## Model 2 

Works in Google Colab and in a local Jupyter Notebook 

In [ ]:
pip install git+https://github.com/tensorflow/docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-ifnm2jor
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-ifnm2jor
  Resolved https://github.com/tensorflow/docs to commit a89b900f18bb647a8a5cba40997eacbe801c0fc0
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=178984 sha256=57b4cceda081431e1c1b53a479ae171eaec5b6425ec7adc82c160653f69cc97d
  Stored in directory: /tmp/pip-ephem-wheel-cache-melx178u/wheels/3b/ee/a2/ab4d36a9a4af495bcb936f3e849d4b497b65fa40548a68d6c3
Successfully built tensorflow-docs


### Import packages

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from datetime import datetime
import tensorboard

import pandas as pd    
import sklearn 
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
#load_ext tensorboard
%reload_ext tensorboard

### Data Loading and Preprocessing

##### **Make shure to follow these steps:**
- Download the Dataset from https://github.com/scfischer/dirk-et-al-2022/tree/main/datasets/

- change the ```dataset_path``` to your own path
- set the ```number of neighbors``` to any number you want to try (remember the number you chose must be number of neighbors +1 (it counts the cell itself)
- 2D Data (DatasetC): Use the code ```distances = cdist(Organoid[['x', 'y']], Organoid[['x', 'y']])``` 
- 3D Data (DatasetD): Use the code  ```distances = cdist(Organoid[['x', 'y', 'z']], Organoid[['x', 'y', 'z']])``` 
- comment out the code you don't need (with #) 

In [ ]:
number_of_neighbors = 8
distances = cdist(Organoid[['x', 'y']], Organoid[['x', 'y']])
#distances = cdist(Organoid[['x', 'y', 'z']], Organoid[['x', 'y', 'z']])

In [ ]:
# Read Organoid Data
dataset_path= '/content/drive/MyDrive/Paper_Cell_reconstruction/Simulated Data/2D_simulated_organoids/Organoids q=0.1.csv'

Data = pd.read_csv(dataset_path,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True)


all_dataframes = []

for i in range(len(Data)):
    
    Organoid = Data[Data['ID'] == i]

    # Compute distances
    distances = distances

    number_of_cells = number_of_neighbors                          # define number of neighbouring cells you want to look at including itself
    nn = np.argsort(distances)                   # argsort 
    ind7 = nn[:,:number_of_cells]                # get index of cell plus its 6 nearest neighbours
    org_fates = np.asarray(Organoid['fate'])     # transform DataFrame to Array

    # Reorganize the arrays such that only the nearest neighbours are included
    dist_pack = np.empty([len(Organoid),number_of_cells])
    fate_pack = np.empty([len(Organoid),number_of_cells])
    for i in range(dist_pack.shape[0]):
        dist_pack[i,:] = distances[i,ind7[i]]
        fate_pack[i,:] = org_fates[ind7[i]]

    # Construct a DataFrame that contains 2 Columns for each of the cell packs
    df = pd.DataFrame()
    for i in range(ind7.shape[1]):
        dist = distances[i,:]
        df['Fate '+str(i)] = fate_pack[:,i]    # initialize first column
        df['Dist '+str(i)] = dist_pack[:,i]    # initialize second column

    all_dataframes.append(df)
    

In [ ]:
#merge the dataset
dataset = pd.concat(all_dataframes, axis=0)
print(dataset)

     Fate 0  Dist 0  Fate 1    Dist 1  Fate 2    Dist 2  Fate 3    Dist 3  \
0       0.0     0.0     1.0  0.753828     1.0  0.801702     1.0  0.861630   
1       0.0     0.0     1.0  1.083788     1.0  1.170211     0.0  1.177150   
2       1.0     0.0     1.0  1.064497     1.0  1.072191     0.0  1.124233   
3       1.0     0.0     0.0  0.773506     0.0  0.804519     1.0  0.892378   
4       1.0     0.0     0.0  0.840429     0.0  0.840879     1.0  0.932006   
..      ...     ...     ...       ...     ...       ...     ...       ...   
130     0.0     0.0     0.0  0.762672     1.0  0.843291     1.0  0.956603   
131     1.0     0.0     0.0  0.695612     0.0  0.695863     1.0  0.813528   
132     0.0     0.0     1.0  0.689279     0.0  0.690729     1.0  0.899236   
133     0.0     0.0     0.0  0.662061     0.0  0.743178     1.0  0.788745   
134     0.0     0.0     1.0  0.727927     0.0  0.981915     1.0  1.213949   

     Fate 4    Dist 4  Fate 5    Dist 5  Fate 6    Dist 6  Fate 7    Dist 7

In [ ]:
#drop the columns we dont need (distance to the cell itself)
# change name of the cellfate column we use as the label
dataset = dataset.drop(columns=['Dist 0'])
dataset=dataset.rename(columns = {'Fate 0':'cellfate'})
print(dataset)

     cellfate  Fate 1    Dist 1  Fate 2    Dist 2  Fate 3    Dist 3  Fate 4  \
0         0.0     1.0  0.753828     1.0  0.801702     1.0  0.861630     0.0   
1         0.0     1.0  1.083788     1.0  1.170211     0.0  1.177150     0.0   
2         1.0     1.0  1.064497     1.0  1.072191     0.0  1.124233     1.0   
3         1.0     0.0  0.773506     0.0  0.804519     1.0  0.892378     0.0   
4         1.0     0.0  0.840429     0.0  0.840879     1.0  0.932006     0.0   
..        ...     ...       ...     ...       ...     ...       ...     ...   
130       0.0     0.0  0.762672     1.0  0.843291     1.0  0.956603     1.0   
131       1.0     0.0  0.695612     0.0  0.695863     1.0  0.813528     1.0   
132       0.0     1.0  0.689279     0.0  0.690729     1.0  0.899236     1.0   
133       0.0     0.0  0.662061     0.0  0.743178     1.0  0.788745     1.0   
134       0.0     1.0  0.727927     0.0  0.981915     1.0  1.213949     1.0   

       Dist 4  Fate 5    Dist 5  Fate 6    Dist 6  

In [ ]:
#here you can have a look at the whole dataset

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(dataset)

     cellfate  Fate 1  Dist 1  Fate 2  Dist 2  Fate 3  Dist 3  Fate 4  Dist 4  \
0         0.0     1.0   0.754     1.0   0.802     1.0   0.862     0.0   0.955   
1         0.0     1.0   1.084     1.0   1.170     0.0   1.177     0.0   1.177   
2         1.0     1.0   1.064     1.0   1.072     0.0   1.124     1.0   1.159   
3         1.0     0.0   0.774     0.0   0.805     1.0   0.892     0.0   0.977   
4         1.0     0.0   0.840     0.0   0.841     1.0   0.932     0.0   0.935   
5         1.0     0.0   0.963     1.0   0.994     0.0   1.038     0.0   1.110   
6         1.0     0.0   0.954     0.0   1.087     0.0   1.100     1.0   1.134   
7         0.0     1.0   1.027     0.0   1.030     1.0   1.035     1.0   1.058   
8         0.0     0.0   0.847     1.0   0.974     1.0   1.059     1.0   1.148   
9         0.0     1.0   1.054     1.0   1.109     0.0   1.115     1.0   1.119   
10        1.0     0.0   0.961     0.0   1.113     0.0   1.130     1.0   1.137   
11        0.0     1.0   1.01

### Split the Dataset into training and test data

In [ ]:
#split the dataset in training data and test data
#its getting split randomly 80% training data, 20% test data
# Use train_test_split() Method.
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)
print(train_dataset)

#split features from label
train_labels = train_dataset.pop('cellfate')
test_labels = test_dataset.pop('cellfate')

     cellfate  Fate 1    Dist 1  Fate 2    Dist 2  Fate 3    Dist 3  Fate 4  \
44        0.0     1.0  1.066347     1.0  1.071704     0.0  1.084519     1.0   
24        0.0     0.0  1.070683     1.0  1.122175     0.0  1.163397     1.0   
95        0.0     1.0  0.897748     0.0  0.900887     0.0  0.903973     1.0   
111       1.0     1.0  0.888151     0.0  0.954901     1.0  0.960725     0.0   
111       1.0     0.0  1.064509     1.0  1.092484     1.0  1.099981     0.0   
..        ...     ...       ...     ...       ...     ...       ...     ...   
117       0.0     0.0  0.868673     0.0  0.907164     1.0  0.987321     1.0   
5         0.0     1.0  1.146602     1.0  1.153987     0.0  1.233690     1.0   
72        0.0     0.0  0.630942     1.0  0.853109     1.0  0.862832     1.0   
75        1.0     0.0  1.122631     0.0  1.128835     1.0  1.146523     1.0   
61        1.0     0.0  1.036140     1.0  1.096274     1.0  1.114878     0.0   

       Dist 4  Fate 5    Dist 5  Fate 6    Dist 6  

### Build the Model with Keras

In [ ]:
#building the model
#sequential model with two densly connected hidden layers

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(2)
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

  model.compile(optimizer = optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model

model = build_model()

#summarize the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                960       
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Dense)            (None, 2)                 130       
                                                                 
Total params: 9,410
Trainable params: 9,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#testing the model on a small batch number

example_batch = train_dataset[:5]
example_result = model.predict(example_batch)
example_result

1/1 [==============================] - 0s 84ms/step


array([[-0.11934292, -0.04270045],
       [-0.0166752 ,  0.0178394 ],
       [ 0.06526291, -0.01864268],
       [-0.09817249, -0.2978792 ],
       [-0.10262328, -0.03085457]], dtype=float32)

### Create a Tensorboard callback with early stopping

In [ ]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = [keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch'),
                        tfdocs.modeling.EpochDots(),
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=75),
]



### Train the Model

In [ ]:

max_epochs = 500

model.fit(
  train_dataset, train_labels,
  epochs=500, validation_split = 0.2, verbose=0,
  callbacks=[tensorboard_callback])


Epoch: 0, accuracy:0.8500,  loss:0.3381,  val_accuracy:0.8968,  val_loss:0.2430,  
....................................................................................................
Epoch: 100, accuracy:0.9493,  loss:0.1079,  val_accuracy:0.9265,  val_loss:0.1819,  
...................

### Evaluate the Model

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

86/86 - 0s - loss: 0.1825 - accuracy: 0.9262 - 226ms/epoch - 3ms/step

Test accuracy: 0.9262235164642334


### Show training on Tensorboard

In [ ]:
%tensorboard --logdir logs